# 1. Configuration

### Imports

In [5]:
import pygad
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import random
import math
import time
from sympy import sympify, symbols, simplify, Eq, solve

### Read the data

In [6]:
dataset = pd.read_csv("dataset.csv")
dataset

,Equation,Xs,Ys
0,((x ** 4) - 6),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-5, 10, 75, 250, 619, 1290, 2395, 4090, 6555,..."
1,(((x / 8) * 2) + 1),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3..."
2,(((x - 1) - 3) / 5),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1...."
3,(x * 5),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60..."
4,(x + 2),"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
...,...,...,...
93,1*x**5 + -2*x**3 + -1*x + -5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-7, 9, 181, 887, 2865, 7333, 16109, 31731, 57..."
94,3*x**5 + 2*x**4 + 1*x + -2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4, 128, 892, 3586, 10628, 25924, 55228, 10650..."
95,-4*x**5 + -1*x + -5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[-10, -135, -980, -4105, -12510, -31115, -6724..."
96,4*x + +1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49,..."


# 2. Representation

In [7]:
maxEquationLength = 25

#### Helper functions

Convert a `string equation` into an `array of tokens`.

In [29]:
def stringEQtoArray(equation):
    arr = np.array([])
    skip = 0

    for index, char in enumerate(equation):
        if skip != 0:
            skip-=1
            continue
    
        if char == ' ':
            continue
        elif char.isdigit():
            if index+1 < len(equation) and equation[index+1].isdigit():
                newIndex = index+1
                while newIndex < len(equation) and equation[newIndex].isdigit():
                    newIndex+=1
                    char = char+equation[newIndex-1]
                    skip += 1
                arr = np.append(arr, char)
            else:
                arr = np.append(arr, char)
        elif char == '-' and equation[index+1].isdigit()::
            if index+1 < len(equation) and equation[index+1].isdigit():
                newIndex = index+1
                while newIndex < len(equation) and equation[newIndex].isdigit():
                    newIndex+=1
                    char = char+equation[newIndex-1]
                    skip += 1
                arr = np.append(arr, char)
            else:
                arr = np.append(arr, char)
        elif char == 'x' or char.isdigit():
            arr = np.append(arr, char)
        elif char == '*' and equation[index+1] == '*':
            arr = np.append(arr, '**')
            skip = 1
        elif char in ['+', '-'] and equation[index+1].isdigit():
            if char == '+':
                arr = np.append(arr, equation[index+1])
            else:
                arr = np.append(arr, char+equation[index+1])
            skip = 1
        elif char in ['+', '-', '*', '/', '(', ')']:
            arr = np.append(arr, char)

    return arr

Convert an `array of tokens` to a `string equation`.

In [9]:
def arrayEQtoString(equation):
    string = ""
    for char in equation:
        string += char
        string += " "
    return string

Convert a string sequance to an `array`.

In [10]:
def stringArrayToArray(string):
    string = string[1:-1]
    arr = string.split(',')
    arr = [float(i) for i in arr]
    arr = np.array(arr)
    return arr

Evaluates `x_values` of the `equation`.

In [11]:
def evaluteEquation(equation, x_values):

    result = np.array([])
    for x in range(1, x_values+1):
        result = np.append(result, eval(equation, {'x': x}, {'math': math}))
    return result

Testing

In [12]:
#vse dela prou :)

for x in range(0,98):
    equation_inputs = dataset.iloc[x].values[0]
    
    outputs = stringArrayToArray(dataset.iloc[x].values[2])
    
    equationOutputs = evaluteEquation(equation_inputs, 100)

    #print(np.sum(outputs - equationOutputs))

    

Tests if the equation `is valid`.

In [13]:
def is_valid(solution):
    #solution is an array of tokens from the equation

    #if there is no x in the equation -> it is not valid
    if 'x' not in solution:
        return False
    
    #if there is no equation -> it is not valid
    if solution.__len__() > maxEquationLength:
        return False

    #if ther eis a digit larger tham 1000 or smaller than -1000 -> it is not valid
    for token in solution:
        if token.isdigit():
            if int(token) > 10 or int(token) < -10:
                return False

    #try to evaluate the equation
    try:
        evaluteEquation(arrayEQtoString(solution), 100)
    except:
        return False

    

    return True

`Simplifies` the equation.

In [14]:
def simplify_equation(equation):
    
    x = symbols('x')

    equationString = arrayEQtoString(equation)

    simp_equation = simplify(equationString)
    #print(simp_equation)
    try:
        evaluteEquation(str(simp_equation), 100)
    except:
        simp_equation = str(equation)
      
    return stringEQtoArray(str(simp_equation))    

Transform an array of `chars` to a `int` array, and it's inverse (`int` -> `char`).

In [15]:
def char_to_int_array(array):
    new_array = []
    for i in array:
        if i == 'x':
            new_array.append(ord(i))
        elif i == '+':
            new_array.append(ord(i))
        elif i == '-':
            new_array.append(ord(i))
        elif i == '*':
            new_array.append(ord(i))
        elif i == '/':
            new_array.append(ord(i))
        elif i == '**':
            new_array.append(420)
        elif i == '(':
            new_array.append(ord(i))
        elif i == ')':
            new_array.append(ord(i))
        else:
            new_array.append(int(float(i)))
    return new_array

In [16]:
def int_array_to_char(array):
    new_array = []
    for i in array:
        if i == ord('x'):
            new_array.append('x')
        elif i == ord('+'):
            new_array.append('+')
        elif i == ord('-'):
            new_array.append('-')
        elif i == ord('*'):
            new_array.append('*')
        elif i == ord('/'):
            new_array.append('/')
        elif i == 420:
            new_array.append('**')
        elif i == ord('('):
            new_array.append('(')
        elif i == ord(')'):
            new_array.append(')')
        else:
            new_array.append(str(i))
    return new_array

Adds `pedding` to the int array, and it's inverse (removes padding).

In [17]:
def add_padding(array):
    new_array = np.zeros(maxEquationLength)
    for i in range(len(array)):
        new_array[i] = array[i]
    return new_array

In [18]:
def remove_padding(array):
    new_array = []
    for i in array:
        if i == 0:
            break
        new_array.append(int(i))
    return new_array

Puts the last two functions together.

In [19]:
def equation_for_GA(equation):
    equation = char_to_int_array(equation)
    equation = add_padding(equation)
    return equation

In [20]:
def equation_from_GA(equation):
    equation = remove_padding(equation)
    equation = int_array_to_char(equation)
    return equation

### 3. Genetic algorithm

In [21]:
#trying for the first equation first
eq_num = 2

true_equation = stringEQtoArray(dataset.iloc[eq_num].values[0])
inputs = stringArrayToArray(dataset.iloc[eq_num].values[1])
outputs = np.array(stringArrayToArray(dataset.iloc[eq_num].values[2]))

print(true_equation)
print(inputs)
print(outputs)

['(' '(' '(' 'x' '1' ')' '3' ')' '/' '5' ')']
[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.
  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.
  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.
  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.
  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.
  99. 100.]
[-0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8  1.   1.2  1.4  1.6  1.8  2.
  2.2  2.4  2.6  2.8  3.   3.2  3.4  3.6  3.8  4.   4.2  4.4  4.6  4.8
  5.   5.2  5.4  5.6  5.8  6.   6.2  6.4  6.6  6.8  7.   7.2  7.4  7.6
  7.8  8.   8.2  8.4  8.6  8.8  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4
 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.  12.2 12.4 12.6 12.8 13.  13.2
 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8 15.  15.2 15.4 15.6 15.8 16.
 16.2 16.4 16.6 16.8 

`Model` returns an array of function outputs and a equation length.

In [25]:
def model(equation):
    #print("modelS")

    equation = equation_from_GA(equation)

    print("MODEL",equation)
    if not is_valid(equation):
        return 0,0

    #rmaybe remove all the () from the equation for the length???
    # equation = [x for x in equation if x != '(' and x != ')']
    # equation = np.array(equation)
    equation_length = len(equation)

    equationString = arrayEQtoString(equation)
    equationOutputs = evaluteEquation(equationString, 100)
            
    return equationOutputs, equation_length


`Fitness function` returns the fitness of a given equation.

In [23]:
def fitness_func(ga_instance, solution, solution_idx):
    #print("FitnesS")
    
    model_outputs,equation_length = model(solution)
    
    if equation_length == 0:
        return -np.inf

    try:

        #change if needed, especialy equation length#

        error = np.sum(np.log10(np.abs(model_outputs - outputs) + 1)) + equation_length*0.1

        if error < 0:
            error = -error

    except:
        return -np.inf

    #print("Fitness: ", -error, "ga_instance.generation: ", ga_instance.generations_completed, "solution_idx: ", solution_idx)

    return -error

Try fitness.

In [28]:
print(true_equation)

print(fitness_func(None, equation_for_GA(true_equation), 0))

['(' '(' '(' 'x' '1' ')' '3' ')' '/' '5' ')']
MODEL [ 40.  40.  40. 120.   1.  41.   3.  41.  47.   5.  41.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
MODEL ['(', '(', '(', 'x', '1', ')', '3', ')', '/', '5', ')']
-inf


`Crossover function` receives  `N parents` and returns ` N children`.

In [21]:
def crossover_func(parents, offspring_size, ga_instance):
    #print("CrossoverS")
    number_of_parents = len(parents)

    array = np.empty((0, maxEquationLength))
    
    for i in range(0, number_of_parents, 2):
        parent1 = sympify(arrayEQtoString(equation_from_GA(parents[i])))
        parent2 = sympify(arrayEQtoString(equation_from_GA(parents[i+1])))

        #print(parent1)
        #print(parent2)

        x = symbols('x')

        subexpressions_parent1 = list(parent1.find(lambda x: x.is_Add or x.is_Mul or x.is_Pow or x.is_Symbol))
        subexpression_to_replace = random.choice(subexpressions_parent1)

        subexpressions_parent2 = list(parent2.find(lambda x: x.is_Add or x.is_Mul or x.is_Pow or x.is_Symbol))
        subexpression_replacement = random.choice(subexpressions_parent2)

        new_equation = parent1.subs(subexpression_to_replace,subexpression_replacement)

        #print(new_equation)

        child = stringEQtoArray(str(new_equation))
        child = simplify_equation(child)
        child = equation_for_GA(child)

        #print(child)

        #add the child array to the array
        child_array = np.array(child, dtype=int).reshape(1, -1)
        array = np.vstack((array, child_array))

    return array  

Try crossover.

In [22]:
parents = [equation_for_GA(stringEQtoArray("x**2 + 2 * x")), equation_for_GA(stringEQtoArray("x**4"))]

print(equation_from_GA(crossover_func(parents, 2, None)[0]))

['x', '**', '4', '*', '(', 'x', '**', '4', '+', '2', ')']


`Mutation function ` receives N equations and returns N `mutated` equations.

In [23]:
def mutation_func(offspring, ga_instance):
    #print("MutationS")
    array = np.empty((0, maxEquationLength))

    for j in range(len(offspring)):

        eq_mutated = equation_from_GA(offspring[j])

        #print("eq_mutated: ", eq_mutated)  
        if not is_valid(eq_mutated):
            #print("Not valid0")
            eq = stringEQtoArray("x + 1")
            offspring_array = np.array(equation_for_GA(eq), dtype=int).reshape(1, -1)
            array = np.vstack((array, offspring_array)).astype(int)
            continue

        #print("Before mutation: ", eq_mutated)

        #Mutation type 1
        if np.random.rand() > 0.2:
            while True:
                
                i = random.randrange(len(eq_mutated))

                # changes a random operator
                if eq_mutated[i] in ['+', '-', '*', '/', '**']:              
                    eq_mutated[i] = np.random.choice(['+', '-', '*', '/', '**'])
                    break
                # changes a random number
                elif eq_mutated[i].isdigit():
                    random_number = str(np.random.randint(-10, 10))
                    if random_number == '0':
                        random_number = '1'
                    eq_mutated[i] = random_number
                    break
                #change the x to ( x + random number )
                elif eq_mutated[i] == 'x':
                    eq_mutated[i] = '('
                    random_number = str(np.random.randint(1, 10))
                    eq_mutated = np.insert(eq_mutated, i+1, ['x', '+', str(random_number), ')'])
                    break
        #Mutation type 2
        else:
            #if it doesnt have () on the start and the end add them
            if eq_mutated[0] != '(' and eq_mutated[-1] != ')':
                eq_mutated = np.append(['('], eq_mutated)
                eq_mutated = np.append(eq_mutated, [')'])

            #add a random operator and a random number on the start or the end
            if np.random.rand() > 0.5:
                #add on the start
                eq_mutated = np.insert(eq_mutated, 0, np.random.choice(['+', '-', '*', '/']))
                random_number = str(np.random.randint(1, 10))
                eq_mutated = np.insert(eq_mutated, 0, random_number)
            else:
                #add on the end
                eq_mutated = np.append(eq_mutated, np.random.choice(['+', '-', '*', '/']))
                random_number = str(np.random.randint(1, 10))
                eq_mutated = np.append(eq_mutated, random_number)

        #print(1,eq_mutated)

        #check if the equation is valid
        if not is_valid(eq_mutated):
            #print("Not valid1")
            eq_mutated = equation_from_GA(offspring[j])

        #print(2,eq_mutated)

        eq_mutated = simplify_equation(eq_mutated)

        #print(3,eq_mutated)

        #check if the equation is valid
        if not is_valid(eq_mutated):
            #print("Not valid2")
            eq_mutated = equation_from_GA(offspring[j])
    
        #print(4,eq_mutated)

        offspring_array = np.array(equation_for_GA(eq_mutated), dtype=int).reshape(1, -1)
        array = np.vstack((array, offspring_array)).astype(int)

    # print("Mutation: ")
    # for i in array:
    #     print(equation_from_GA(i))

    return array

Try mutation.

In [24]:
equation_from_GA(mutation_func(np.array([equation_for_GA(['x','+','1'])]), 1)[0])

['x', '+', '5']

Initialize the population.

In [25]:
pop1 = [['x','+', '1']] + [['x','+', '2']] +[['x','+', '3']] + [['x','+', '4']] + [['x','+', '5']] +[['x','+', '6']] + [['x','+', '7']] + [['x','+', '8']] + [['x','+', '9']]
pop2 = [['x','*', '1']] + [['x','*', '2']] +[['x','*', '3']] + [['x','*', '4']] + [['x','*', '5']]  + [['x','*', '6']] +[['x','*', '7']] + [['x','*', '8']] + [['x','*', '9']]
pop3 = [['x','**', '2']] +[['x','**', '3']] + [['x','**', '4']] + [['x','**', '5']] + [['x','**', '6']] +[['x','**', '7']] + [['x','**', '8']] + [['x','**', '9']]
pop4 = [['x','/', '2']] +[['x','/', '3']] + [['x','/', '4']] + [['x','/', '5']] + [['x','/', '6']] +[['x','/', '7']] + [['x','/', '8']] + [['x','/', '9']]

initial_population = pop1 + pop2 + pop3 + pop4

#transform the initial population to a int array
initial_population = [equation_for_GA(i) for i in initial_population]


tmp1 = np.array([i for i in range(1, 10)])
tmp2 = np.array([i for i in range(-10, 0)])
gene_space = np.array([ord('x'), ord('+'), ord('-'), ord('*'), ord('/'), 420]) 
gene_space = np.append(gene_space, tmp1)
gene_space = np.append(gene_space, tmp2)

In [26]:
sol_per_pop = initial_population.__len__()
num_parents_mating = 20
keep_elitism = int(sol_per_pop - num_parents_mating/2)

print("sol_per_pop: ", sol_per_pop)
print("num_parents_mating: ", num_parents_mating)
print("keep_elitism: ", keep_elitism)

sol_per_pop:  34
num_parents_mating:  20
keep_elitism:  24


In [27]:
def printCrossover(x,y):
    print("Crossover")

def printMutation(x,y):
    print("Mutation")

def printFitness(x,y):
    print("Fitness")

def printParents(y,parents):
    print("Parents")

In [28]:
for i in range(20):   
    try:
        print("Equation: ", i)

        true_equation = stringEQtoArray(dataset.iloc[i].values[0])
        inputs = stringArrayToArray(dataset.iloc[i].values[1])
        outputs = np.array(stringArrayToArray(dataset.iloc[i].values[2]))


        ga_instance = pygad.GA(num_generations=100,
                            num_parents_mating=num_parents_mating,
                            fitness_func=fitness_func,
                            initial_population=initial_population,
                            gene_type=int,
                            parent_selection_type="tournament",
                            keep_elitism=keep_elitism,
                            crossover_probability=0.8,
                            crossover_type=crossover_func,
                            mutation_type=mutation_func,
                            mutation_probability=0.5,
                            gene_space=gene_space,
                            stop_criteria="saturate_50")
                            # on_crossover=printCrossover,
                            # on_mutation=printMutation,
                            # on_fitness=printFitness,
                            # on_parents=printParents)

        ga_instance.run()

        print(ga_instance.plot_fitness())

        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        print("Parameters of the best solution : {solution}".format(solution=solution))
        print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))


        x = range(1, 101)

        # Assuming you have two output arrays of the same length
        y1 = outputs

        y2 = evaluteEquation(arrayEQtoString(equation_from_GA(solution)), 100)

        print("True equation: ", true_equation)
        print("Predicted equation: ", equation_from_GA(solution))

        # Plotting the first array
        plt.plot(x, y1, label='True equation')

        # Plotting the second array
        plt.plot(x, y2, label='Predicted equation')

        # Adding labels and title
        plt.xlabel('X-axis label')
        plt.ylabel('Y-axis label')
        plt.title('Two Arrays Plot')

        # Adding legend
        plt.legend()

        # Display the plot
        plt.show()

    except Exception as e:
        print("ERROR: ", e)
        continue


Equation:  0
